## Create nice images for instagram, programmatically


Necessities:

* Image, that still needs to be sized, cropped and positioned into an 1080 by 1080 square
* Transparrent dark bottom layer, place on top of the image so that the text stands out more.
* Logo, needs to be placed on top in the bottom left corner
* 2x text, for the place and for the country.

To make our life more easier, we combined the transparant bottom and logo into a single layer.

Implementation moved to module `stairway.instagram.image`.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import requests
from io import BytesIO

from stairway.instagram.image import create_instagram_image, resize_image, crop_image_center

### Init

This notebook demos how the function works. First pick a pic to format.

In [ ]:
input_folder = '../../data/instagram-post/'

# place specifics
place_id = 644651
place_id = 627255  # a smaller than 1080x1080 example
place_id = 220380
place_id = 482586
place_image  = input_folder + f'{place_id}.jpg' 
# place_image = input_folder + 'portrait_example.jpg'

# or retriev via url
# TODO: make these dynamic (depending on the place_id)
place_url_b = 'https://farm3.staticflickr.com/2601/3970594887_fbdbd8dbaa_b.jpg'  # large size
# place_url_o = 'https://live.staticflickr.com/2601/3970594887_6cd6d05565_o.jpg'  # original size (takes 6x longer)

# place data
data_dir = '../../data/wikivoyage/enriched/'
file_name = 'wikivoyage_destinations.csv'
place_df = pd.read_csv(data_dir + file_name).set_index("id", drop=False)

Get text for on image from the database

In [ ]:
place_text   = place_df.loc[place_id]["name"]
country_text = place_df.loc[place_id]["country"]

## Image processing

### Single function call

In [ ]:
# create_instagram_image(place_image, place_text, country_text, input_folder + 'test.jpeg')
create_instagram_image(place_url_b, place_text, country_text, input_folder + 'test.jpeg')

#### Parralel processing

Implementation moved to script `instagram_image_processing.py`

### Image Processing - Step by Step

Follow this [blog](https://auth0.com/blog/image-processing-in-python-with-pillow) which has very nice example code and explanation on how to do most of the beneath steps.

Initiate the image from file:

In [ ]:
image = Image.open(place_image)
 
image.size

Or initiate from url:

In [ ]:
# response = requests.get(place_url_o)
# image = Image.open(BytesIO(response.content))

# image.size

First resize. `.thumbnail()` can only make images smaller, so resize manually by calculating the aspect ratio first. This way we can also process images that have dimensions smaller than 1080x1080.

In [ ]:
image = resize_image(image)

image.size

Crop center

In [ ]:
image = crop_image_center(image)

image.size

Paste logo layer on top

In [ ]:
logo = Image.open(logo_layer)

# don't need to provide position as both are already 1080x1080
image.paste(logo, mask=logo)

Place text. Following this [tutorial](https://haptik.ai/tech/putting-text-on-image-using-python/)

In [ ]:
add_text(image, place_text, (210, 870), place_font, 80)
add_text(image, country_text, (210, 962), country_font, 40)

Ready!

In [ ]:
image

Save if wanted.

In [ ]:
image.save(input_folder + 'output.jpeg', optimize=True, quality=90)

Done.